# Data Cleaning
 Our source dataset contains the temperature data for 3,448 cities from 1743 to 2013. We will filter out temperature information from 1900 onwards and select the data from only the top 50 cities with the highest GDP in the world.

## 1. Read from source dataset and extract data from 1900 to 2013
Source: https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data

In [2]:
import pandas as pd

In [3]:
temp_city_df = pd.read_csv("Earth Surface Temperature Data - Kaggle/GlobalLandTemperaturesByCity.csv",
                               parse_dates=['dt'],
                               index_col=['dt'])

In [4]:
temp_city_df.head()

,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
dt,,,,,,
1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [5]:
temp_city_df = temp_city_df[temp_city_df.index > '1900-01-01']

## 2. Read the list of the top 50 cities in the world with highest GDP
Source: http://www.citymayors.com/statistics/richest-cities-2020.html

In [6]:
top_gdp_world_city_df = pd.read_csv("Data/World City GDP/WorldCityGDP2020.csv",
                                       index_col=['Rank'])

In [7]:
top_gdp_world_city_df.head(50)['City'].values

array(['Tokyo', 'New York', 'Los Angeles', 'London', 'Chicago', 'Paris',
       'Mexico City', 'Philadelphia', 'Osaka', 'Washington DC',
       'Buenos Aires', 'Boston', 'Sao Paulo', 'Hong Kong', 'Dallas',
       'Shanghai', 'Seoul', 'Atlanta', 'San Francisco', 'Houston',
       'Miami', 'Toronto', 'Moscow', 'Mumbai', 'Madrid', 'Detroit',
       'Istanbul', 'Seattle', 'Beijing', 'Metro Manila', 'Rio de Janeiro',
       'Sydney', 'Jakarta', 'Delhi', 'Phoenix', 'Guangzhou',
       'Minneapolis', 'Kolkata', 'San Diego', 'Singapore', 'Cairo',
       'Barcelona', 'Melbourne', 'Denver', 'Rome', 'Bangkok', 'Montreal',
       'Milan', 'Tehran', 'Riyadh'], dtype=object)

## 3. Join the 2 dataframes to get desired data with the temperature information of top 50 cities

### 3.1 Look for any missing cities

In [8]:
top_gdp_cities_in_overall_list =  temp_city_df[temp_city_df['City'].isin(top_gdp_world_city_df.head(50)['City'])]['City'].unique()
top_gdp_cities_in_overall_list

array(['Atlanta', 'Bangkok', 'Barcelona', 'Boston', 'Cairo', 'Chicago',
       'Dallas', 'Delhi', 'Denver', 'Detroit', 'Guangzhou', 'Houston',
       'Istanbul', 'Jakarta', 'London', 'Los Angeles', 'Madrid',
       'Melbourne', 'Miami', 'Milan', 'Minneapolis', 'Montreal', 'Moscow',
       'New York', 'Paris', 'Philadelphia', 'Phoenix', 'Riyadh', 'Rome',
       'San Diego', 'San Francisco', 'Seattle', 'Seoul', 'Shanghai',
       'Singapore', 'Sydney', 'Tokyo', 'Toronto'], dtype=object)

In [9]:
len(top_gdp_cities_in_overall_list)

38

In [10]:
top_gdp_world_city_df[~top_gdp_world_city_df['City'].isin(top_gdp_cities_in_overall_list)]

,City,Country,GDP_Billion_$
Rank,,,
7,Mexico City,Mexico,608
9,Osaka,Japan,430
10,Washington DC,USA,426
11,Buenos Aires,Argentina,416
13,Sao Paulo,Brazil,411
14,Hong Kong,China,407
24,Mumbai,India,300
29,Beijing,China,259
30,Metro Manila,Philippines,257


#### We are missing 12 cities, mostly due to naming inconsistencies.

### 3.2 Change the city names to permit join

In [11]:
top_gdp_world_city_df.replace('Mexico City','Mexico', inplace=True)
top_gdp_world_city_df.replace('Osaka','Kobe', inplace=True)
top_gdp_world_city_df.replace('Washington DC','Washington', inplace=True)
top_gdp_world_city_df.replace('Buenos Aires','La Plata', inplace=True)
top_gdp_world_city_df.replace('Sao Paulo','São Paulo', inplace=True)
top_gdp_world_city_df.replace('Mumbai','Bombay', inplace=True)
top_gdp_world_city_df.replace('Beijing','Peking', inplace=True)
top_gdp_world_city_df.replace('Metro Manila','Manila', inplace=True)
top_gdp_world_city_df.replace('Rio de Janeiro','Rio De Janeiro', inplace=True)
top_gdp_world_city_df.replace('Kolkata','Calcutta', inplace=True)

In [12]:
top_gdp_cities_in_overall_list =  temp_city_df[temp_city_df['City'].isin(top_gdp_world_city_df.head(50)['City'])]['City'].unique()
print(len(top_gdp_cities_in_overall_list))
top_gdp_world_city_df[~top_gdp_world_city_df['City'].isin(top_gdp_cities_in_overall_list)]

48


,City,Country,GDP_Billion_$
Rank,,,
14,Hong Kong,China,407
49,Tehran,Iran,172


#### Still 2 more cities are missing. We will try to use the geolocation information to identify the names used for Hong Kong and Tehran

### 3.3 Filter dataset by geolocation to identify remaining missing cities

#### Searching for Hong Kong located at 22.3193° N, 114.1694° E

In [13]:
temp_city_df[(temp_city_df['Latitude'].str.rstrip('N,S').astype(float) > 21) & 
        (temp_city_df['Longitude'].str.rstrip('E,W').astype(float) > 113) &
        (temp_city_df['Latitude'].str.rstrip('N,S').astype(float) < 23) & 
        (temp_city_df['Longitude'].str.rstrip('E,W').astype(float) < 115)]['City'].unique()

array(['Sanbu', 'Taicheng', 'Zhongshan', 'Zhuhai'], dtype=object)

#### Searching for Tehran located at 35.7248° N, 51.3816° E

In [14]:
temp_city_df[(temp_city_df['Latitude'].str.rstrip('N,S').astype(float) > 34) & 
        (temp_city_df['Longitude'].str.rstrip('E,W').astype(float) > 50) &
        (temp_city_df['Latitude'].str.rstrip('N,S').astype(float) < 37) & 
        (temp_city_df['Longitude'].str.rstrip('E,W').astype(float) < 53)]['City'].unique()

array(['Amol', 'Arak', 'Babol', 'Eslamshahr', 'Karaj', 'Kashan',
       'Qarchak', 'Qom', 'Saveh', 'Varamin'], dtype=object)

#### It seems that our source dataset does not contain information of Hong Kong and Tehran. Since we have identiefied nearby cities,and since they will exhibit same climate behavior, we will use those nearby cities instead.

In [15]:
top_gdp_world_city_df.replace('Hong Kong','Zhuhai', inplace=True)
top_gdp_world_city_df.replace('Tehran','Eslamshahr', inplace=True)

top_gdp_cities_in_overall_list =  temp_city_df[temp_city_df['City'].isin(top_gdp_world_city_df.head(50)['City'])]['City'].unique()
print(len(top_gdp_cities_in_overall_list))
top_gdp_world_city_df[~top_gdp_world_city_df['City'].isin(top_gdp_cities_in_overall_list)]

50


,City,Country,GDP_Billion_$
Rank,,,


In [16]:
top_gdp_cities_in_overall_list

array(['Atlanta', 'Bangkok', 'Barcelona', 'Bombay', 'Boston', 'Cairo',
       'Calcutta', 'Chicago', 'Dallas', 'Delhi', 'Denver', 'Detroit',
       'Eslamshahr', 'Guangzhou', 'Houston', 'Istanbul', 'Jakarta',
       'Kobe', 'La Plata', 'London', 'Los Angeles', 'Madrid', 'Manila',
       'Melbourne', 'Mexico', 'Miami', 'Milan', 'Minneapolis', 'Montreal',
       'Moscow', 'New York', 'Paris', 'Peking', 'Philadelphia', 'Phoenix',
       'Rio De Janeiro', 'Riyadh', 'Rome', 'São Paulo', 'San Diego',
       'San Francisco', 'Seattle', 'Seoul', 'Shanghai', 'Singapore',
       'Sydney', 'Tokyo', 'Toronto', 'Washington', 'Zhuhai'], dtype=object)

In [17]:
top_50_world_city_temp_df = temp_city_df[temp_city_df['City'].isin(top_gdp_cities_in_overall_list)]

In [18]:
top_50_world_city_temp_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 72292 entries, 1900-02-01 to 2013-09-01
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   AverageTemperature             72260 non-null  float64
 1   AverageTemperatureUncertainty  72260 non-null  float64
 2   City                           72292 non-null  object 
 3   Country                        72292 non-null  object 
 4   Latitude                       72292 non-null  object 
 5   Longitude                      72292 non-null  object 
dtypes: float64(2), object(4)
memory usage: 3.9+ MB


### 3.4 Riverting the edited city names back to original

In [19]:
top_50_world_city_temp_df.replace('Mexico', 'Mexico City', inplace=True)
top_50_world_city_temp_df.replace('Kobe', 'Osaka', inplace=True)
top_50_world_city_temp_df.replace('Washington', 'Washington DC', inplace=True)
top_50_world_city_temp_df.replace('La Plata', 'Buenos Aires', inplace=True)
top_50_world_city_temp_df.replace('São Paulo', 'Sao Paulo', inplace=True)
top_50_world_city_temp_df.replace('Bombay', 'Mumbai', inplace=True)
top_50_world_city_temp_df.replace('Peking', 'Beijing', inplace=True)
top_50_world_city_temp_df.replace('Rio De Janeiro', 'Rio de Janeiro',inplace=True)
top_50_world_city_temp_df.replace('Calcutta', 'Kolkata', inplace=True)
top_50_world_city_temp_df.replace('Zhuhai', 'Hong Kong', inplace=True)
top_50_world_city_temp_df.replace('Eslamshahr', 'Tehran', inplace=True)

/var/folders/l_/pnx7zdd15nx_h4zdjl1t3hpm0000gn/T/ipykernel_54879/285201191.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_50_world_city_temp_df.replace('Mexico', 'Mexico City', inplace=True)
/var/folders/l_/pnx7zdd15nx_h4zdjl1t3hpm0000gn/T/ipykernel_54879/285201191.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_50_world_city_temp_df.replace('Kobe', 'Osaka', inplace=True)
/var/folders/l_/pnx7zdd15nx_h4zdjl1t3hpm0000gn/T/ipykernel_54879/285201191.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.o

#### We have obtained our required cleaned dataset

## 4. Export data to csv

In [20]:
top_50_world_city_temp_df.to_csv("Data/Cleaned/top_50_world_city_temp.csv")